# Лабораторная работа №1

## Изучение симплекс-метода решения задачи ЛП

## Условие задачи

Требуется найти решение следующей задачи ЛП:

$$F=\textbf{cx}\rightarrow max,$$
$$\textbf{Ax}\leq\textbf{b},$$
$$\textbf{x}\geq 0.$$

In [1]:
import numpy as np

from IPython.display import display_markdown

from src.latex import array_to_latex, matrix_to_latex, equation_system, equation_body, check, Brackets


# Source data
c = np.array([3, 1, 4], dtype=np.float32)
A = np.array([2, 1, 1, 1, 4, 0, 0, 0.5, 1], dtype=np.float32).reshape(3, 3)
b = np.array([6, 4, 1], dtype=np.float32)

# c = np.array([7, 5, 3], dtype=np.float32)
# A = np.array([1, 2, 0, 4, 1, 1, 0, 0.5, 1], dtype=np.float32).reshape(3, 3)
# b = np.array([30, 40, 50], dtype=np.float32)

display_markdown(
    "<center> <h2> Исходные данные согласно Варианту 2 </h2> </center>",
    array_to_latex(c, 'c', Brackets.square),
    matrix_to_latex(A, 'A', Brackets.square),
    array_to_latex(b, 'b', Brackets.square),
    raw=True
)

<center> <h2> Исходные данные согласно Варианту 2 </h2> </center>

$c = \begin{bmatrix}
3.0 & 1.0 & 4.0
\end{bmatrix}$

$A = \begin{bmatrix}
2.0 & 1.0 & 1.0\\
1.0 & 4.0 & 0.0\\
0.0 & 0.5 & 1.0
\end{bmatrix}$

$b = \begin{bmatrix}
6.0 & 4.0 & 1.0
\end{bmatrix}$

In [2]:
system = np.hstack([A, np.eye(A.shape[0])])
labels = [f"x_{idx + 1}" for idx in range(system.shape[1])]

eq = equation_body(c, labels[:A.shape[1]])
eq = f"$$F=-({eq}) \\rightarrow min$$"
cond = f"$${', '.join(labels)} \geq 0$$"

display_markdown(
    "<center> <h2> Задача ЛП в канонической форме </h2> </center>",
    eq, equation_system(system, b, labels), cond,
    raw=True
)

<center> <h2> Задача ЛП в канонической форме </h2> </center>

$$F=-(3.0x_1+x_2+4.0x_3) \rightarrow min$$

$\begin{cases}2.0x_1+x_2+x_3+x_4=6.0\\
x_1+4.0x_2+x_5=4.0\\
0.5x_2+x_3+x_6=1.0
\end{cases}$

$$x_1, x_2, x_3, x_4, x_5, x_6 \geq 0$$

## Выполнение работы

In [3]:
from IPython.display import display_markdown

from src.simplex import Simplex

result, solvable = Simplex.resolve(c, A, b)


if solvable:
    tgt, base = result.function_to_md(inverse=True)

    display_markdown(
        "## Результаты работы simplex-алгоритма",
        result.table_to_md(),
        "### Целевая функция (для исходной задачи)",
        tgt,
        raw=True
    )
else:
    display_markdown(
        "## Результаты работы simplex-алгоритма",
        result.table_to_md(),
        "### Задача не может быть разрешена",
        raw=True
    ) 

## Результаты работы simplex-алгоритма

|       |   $s_0$ |   $x_4$ |   $x_2$ |   $x_6$ |
|:------|--------:|--------:|--------:|--------:|
| $x_1$ |     2.5 |     0.5 |    0.25 |    -0.5 |
| $x_5$ |     1.5 |    -0.5 |    3.75 |     0.5 |
| $x_3$ |     1   |    -0   |    0.5  |     1   |
| $F$   |   -11.5 |    -1.5 |   -1.75 |    -2.5 |

### Целевая функция (для исходной задачи)

$F=11.5$

In [4]:
from src.simplex import Simplex


def resolve(c, A, b):
    gen = Simplex.resolve_gen(c, A, b)

    prev, _, _ = next(gen)

    tgt, base = prev.function_to_md()

    display_markdown(
        "### Изначальная симплекс-таблица",
        prev.table_to_md(),
        raw=True
    )

    table, pos, flag = next(gen)

    if pos and flag:
        display_markdown(
            "### Недопустимое решение",
            base,
            f"### Разрешающий элемент $x_{{{pos[0], pos[1]}}}={prev.table[pos]}$",
            "### Исправленная симплекс-таблица",
            table.table_to_md(),
            raw=True
        )
    elif not pos and not flag:
        display_markdown(
            "### Недопустимое решение",
            base,
            f"### Невозможно найти разрешающий элемент",
            raw=True
        )

        return

    prev = table
    for idx, (table, pos, flag) in enumerate(gen):
        tgt, base = table.function_to_md()

        display_markdown(
            f"### {idx + 1} шаг. Разрешающий элемент $x_{{{pos[0]},{pos[1]}}}={prev.table[pos]}$",
            table.table_to_md(),
            "### Опорное решение",
            base,
            "### Целевая функция",
            tgt,
            raw=True
        )

        prev = table
    
    tgt, base = table.function_to_md(inverse=True)

    display_markdown(
        f"### Результирующая симплекс-таблица",
        table.table_to_md(),
        "### Целевая функция для исходной задачи",
        tgt,
        raw=True
    )

resolve(c, A, b)

### Изначальная симплекс-таблица

|       |   $s_0$ |   $x_1$ |   $x_2$ |   $x_3$ |
|:------|--------:|--------:|--------:|--------:|
| $x_4$ |       6 |       2 |     1   |       1 |
| $x_5$ |       4 |       1 |     4   |       0 |
| $x_6$ |       1 |       0 |     0.5 |       1 |
| $F$   |       0 |       3 |     1   |       4 |

### 1 шаг. Разрешающий элемент $x_{0,1}=0.5$

|       |   $s_0$ |   $x_4$ |   $x_2$ |   $x_3$ |
|:------|--------:|--------:|--------:|--------:|
| $x_1$ |       3 |     0.5 |     0.5 |     0.5 |
| $x_5$ |       1 |    -0.5 |     3.5 |    -0.5 |
| $x_6$ |       1 |    -0   |     0.5 |     1   |
| $F$   |      -9 |    -1.5 |    -0.5 |     2.5 |

### Опорное решение

...

$F=-11.5$

### Результирующая симплекс-таблица

|       |   $s_0$ |   $x_4$ |   $x_2$ |   $x_6$ |
|:------|--------:|--------:|--------:|--------:|
| $x_1$ |     2.5 |     0.5 |    0.25 |    -0.5 |
| $x_5$ |     1.5 |    -0.5 |    3.75 |     0.5 |
| $x_3$ |     1   |    -0   |    0.5  |     1   |
| $F$   |   -11.5 |    -1.5 |   -1.75 |    -2.5 |

### Целевая функция для исходной задачи

$F=11.5$

In [6]:
labels = {
    f'x_{i + 1}' for i, _ in enumerate(result.h_labels[1:])
}

values = dict()
for value, label in zip(result.table[:-1, 0].flatten(), result.v_labels):
    if label not in labels:
        continue

    values[label] = value

for label in labels.difference(values.keys()):
    values[label] = 0

labels, values = map(
    list, zip(
        *sorted(values.items(), key=lambda x: x[0])
    )
)

display_markdown(
    "### Проверка решения",
    f"$F={equation_body(values, labels)}={check(values, c)}$",
    raw=True
)

### Проверка решения

$F=2.5x_1+x_3=2.5\cdot3.0+1.0\cdot4.0=11.5$